# Python Algorithmic Trading Cookbook

## Chapter 10: Algorithmic Trading: Paper Trading

This Jupyter Notebook is created using Python version 3.8.2

----

### Recipe 1: EMA-Regular-Order Strategy: Fetching the Strategy

In [82]:
from pyalgotrading.algobulls import AlgoBullsConnection

In [83]:
algobulls_connection = AlgoBullsConnection()

In [84]:
algobulls_connection.get_authorization_url()

Please login to this URL with your AlgoBulls credentials and get your developer access token: https://app.algobulls.com/user/login


'https://app.algobulls.com/user/login'

In [85]:
algobulls_connection.set_access_token('7edd6cea07cb0f2549dae7d42115c130140417dc')

In [86]:
all_strategies = algobulls_connection.get_all_strategies()
all_strategies

,strategyCode,strategyName
0,49287246f9704bbcbad76ade9e2091d9,EMA Regular Order Strategy
1,4faf514fe096432b8e9f80f5951bd2ea,MACD Bracket Order Strategy


In [87]:
strategy_code1 = all_strategies.iloc[0]['strategyCode']
strategy_code1

'49287246f9704bbcbad76ade9e2091d9'

In [88]:
strategy_details1 = algobulls_connection.get_strategy_details(strategy_code1)
print(strategy_details1)

class StrategyEMARegularOrder(StrategyBase):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.timeperiod1 = self.strategy_parameters['timeperiod1']
        self.timeperiod2 = self.strategy_parameters['timeperiod2']

        self.main_order = None

    def initialize(self):
        self.main_order = {}

    @staticmethod
    def name():
        return 'EMA Regular Order Strategy'

    @staticmethod
    def versions_supported():
        return AlgoBullsEngineVersion.VERSION_3_2_0

    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        ema_x = talib.EMA(hist_data['close'], timeperiod=self.timeperiod1)
        ema_y = talib.EMA(hist_data['close'], timeperiod=self.timeperiod2)
        crossover_value = self.utils.crossover(ema_x, ema_y)
        return crossover_value

    def strategy_select_instruments_for_entry(self, candle, instruments_bucket):

        selected_instruments_bucket = 

### Recipe 2: EMA-Regular-Order Strategy: Paper Trading the Strategy

In [89]:
from datetime import time
from pyalgotrading.constants import *

In [90]:
instruments = algobulls_connection.search_instrument('SBIN')
instruments

[{'id': 7, 'value': 'NSE:SBIN'}]

In [91]:
instrument = instruments[0]['value']
instrument

'NSE:SBIN'

In [92]:
algobulls_connection.papertrade(strategy_code=strategy_code1, 
                                start_time=time(hour=9, minute=15), 
                                end_time=time(hour=15, minute=30), 
                                instrument=instrument, 
                                lots=1,
                                strategy_parameters={
                                    'timeperiod1': 5,
                                    'timeperiod2': 12
                                }, 
                                candle_interval=CandleInterval.MINUTES_15)

Setting Strategy Config... Success.
Submitting PAPERTRADING job... Success.


In [56]:
algobulls_connection.get_papertrading_job_status(strategy_code1)

{'data': 'STARTING'}

In [57]:
algobulls_connection.get_papertrading_job_status(strategy_code1)

{'data': 'STARTING'}

### There's more...

In [58]:
algobulls_connection.stop_papertrading_job(strategy_code1)

Stopping PAPERTRADING job... Fail.
Forbidden: {'message': 'Strategy execution is in the STARTING state. Please wait for it to go to the STARTED state before STARTING it.'}


In [59]:
algobulls_connection.get_papertrading_job_status(strategy_code1)

{'data': 'STARTING'}

In [60]:
algobulls_connection.get_papertrading_job_status(strategy_code1)

{'data': 'STARTING'}

### Recipe 3: EMA-Regular-Order Strategy: Fetching Paper Trading Logs in Realtime

In [61]:
logs = algobulls_connection.get_papertrading_logs(strategy_code1)
print(logs)

[2020-07-31 09:09:39] Performing sanity checks on cfg strategy_parameters, setting up broker connection and required data structures...
[2020-07-31 09:09:39] ABBroker connection has been setup successfully.
[2020-07-31 09:09:39] Sanity checks on cfg successful.
[2020-07-31 09:09:39] Setting up broker connection...
[2020-07-31 09:09:39] Broker connection has been setup successfully.
[2020-07-31 09:09:39] (NSE_EQ) Funds available in client's ABVirtualBroker account is : Rs. '1000000000.00'
[2020-07-31 09:09:39] 
########################################
 INITIALIZING ALGOBULLS CORE (v3.2.0 SECURE MODE)... 
########################################
[2020-07-31 09:09:39] Welcome ALGOBULLS VIRTUAL USER!
[2020-07-31 09:09:39] Reading strategy...
[2020-07-31 09:09:39] Entering Paper Trading mode. Henceforth, all timestamps will be Paper Trading timestamps...
[2020-07-31 09:09:39] STARTING ALGOBULLS CORE...
[2020-07-31 09:09:39] 
            
  #####  #######    #    ######  ####### ### #     # 

In [62]:
logs = algobulls_connection.get_papertrading_logs(strategy_code1)
print(logs)

[2020-07-31 09:09:39] Performing sanity checks on cfg strategy_parameters, setting up broker connection and required data structures...
[2020-07-31 09:09:39] ABBroker connection has been setup successfully.
[2020-07-31 09:09:39] Sanity checks on cfg successful.
[2020-07-31 09:09:39] Setting up broker connection...
[2020-07-31 09:09:39] Broker connection has been setup successfully.
[2020-07-31 09:09:39] (NSE_EQ) Funds available in client's ABVirtualBroker account is : Rs. '1000000000.00'
[2020-07-31 09:09:39] 
########################################
 INITIALIZING ALGOBULLS CORE (v3.2.0 SECURE MODE)... 
########################################
[2020-07-31 09:09:39] Welcome ALGOBULLS VIRTUAL USER!
[2020-07-31 09:09:39] Reading strategy...
[2020-07-31 09:09:39] Entering Paper Trading mode. Henceforth, all timestamps will be Paper Trading timestamps...
[2020-07-31 09:09:39] STARTING ALGOBULLS CORE...
[2020-07-31 09:09:39] 
            
  #####  #######    #    ######  ####### ### #     # 

### Recipe 4: EMA-Regular-Order Strategy: Fetching Paper Trading Report - Profit-&-Loss Table

In [63]:
algobulls_connection.get_papertrading_report_pnl_table(strategy_code1)

Report not available yet. Please retry in sometime


### Recipe 5: EMA-Regular-Order Strategy: Fetching Paper Trading Report - Statistics Table

In [64]:
algobulls_connection.get_papertrading_report_statistics(strategy_code1)

Report not available yet. Please retry in sometime


### Recipe 6: EMA-Regular-Order Strategy: Fetching Paper Trading Report - Order History

In [65]:
order_history = algobulls_connection.get_papertrading_report_order_history(strategy_code1)
print(order_history)

[2020-07-31 14:51:57] Order History not available yet. Please retry in sometime.


### Recipe 7: MACD-Bracket-Order Strategy: Fetching the Strategy

In [66]:
from pyalgotrading.algobulls import AlgoBullsConnection

In [67]:
algobulls_connection = AlgoBullsConnection()

In [68]:
algobulls_connection.get_authorization_url()

Please login to this URL with your AlgoBulls credentials and get your developer access token: https://app.algobulls.com/user/login


'https://app.algobulls.com/user/login'

In [70]:
algobulls_connection.set_access_token('b4e0f923d085d48f4665a079d760d2bb94c33a94')

In [71]:
all_strategies = algobulls_connection.get_all_strategies()
all_strategies

,strategyCode,strategyName
0,49287246f9704bbcbad76ade9e2091d9,EMA Regular Order Strategy
1,4faf514fe096432b8e9f80f5951bd2ea,MACD Bracket Order Strategy


In [72]:
strategy_code2 = all_strategies.iloc[1]['strategyCode']
strategy_code2

'4faf514fe096432b8e9f80f5951bd2ea'

In [73]:
strategy_details2 = algobulls_connection.get_strategy_details(strategy_code2)
print(strategy_details2)

class StrategyMACDBracketOrder(StrategyBase):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.fastMA_period = self.strategy_parameters['fastma_period']
        self.slowMA_period = self.strategy_parameters['slowma_period']
        self.signal_period = self.strategy_parameters['signal_period']
        self.stoploss = self.strategy_parameters['stoploss_trigger']
        self.target = self.strategy_parameters['target_trigger']
        self.trailing_stoploss = self.strategy_parameters['trailing_stoploss_trigger']

        self.main_order = None

    def initialize(self):
        self.main_order = {}

    @staticmethod
    def name():
        return 'MACD Bracket Order Strategy'

    @staticmethod
    def versions_supported():
        return AlgoBullsEngineVersion.VERSION_3_2_0

    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        macdline, macdsignal, _ = talib.MACD(hist_data['close

### Recipe 8: MACD-Bracket-Order Strategy: Paper Trading the Strategy

In [74]:
from datetime import time
from pyalgotrading.constants import *

In [75]:
instrument = algobulls_connection.search_instrument('TATASTEEL')
instrument

[{'id': 1, 'value': 'NSE:TATASTEEL'}]

In [76]:
instrument = instruments[0]['value']
instrument

'NSE:SBIN'

In [35]:
algobulls_connection.papertrade(strategy_code=strategy_code2, 
                                start_time=time(hour=9, minute=15), 
                                end_time=time(hour=15, minute=30), 
                                instrument=instrument,
                                lots=1,
                                strategy_parameters={
                                    'fastma_period': 26, 
                                    'slowma_period': 6, 
                                    'signal_period': 9,
                                    'target_trigger': 0.01, 
                                    'stoploss_trigger': 0.01, 
                                    'trailing_stoploss_trigger': 1
                                },
                                candle_interval=CandleInterval.MINUTES_15)

Setting Strategy Config... Success.
Submitting PAPERTRADING job... Success.


In [36]:
algobulls_connection.get_papertrading_job_status(strategy_code2)

{'data': 'STARTING'}

In [37]:
algobulls_connection.get_papertrading_job_status(strategy_code2)

{'data': 'STARTING'}

### There's more...

In [77]:
algobulls_connection.stop_papertrading_job(strategy_code2)

Stopping PAPERTRADING job... Fail.
Forbidden: {'message': 'Strategy execution is in the STARTING state. Please wait for it to go to the STARTED state before STARTING it.'}


In [39]:
algobulls_connection.get_papertrading_job_status(strategy_code2)

{'data': 'STARTING'}

In [40]:
algobulls_connection.get_papertrading_job_status(strategy_code2)

{'data': 'STARTING'}

### Recipe 9: MACD-Bracket-Order Strategy: Fetching Paper Trading Logs in Realtime

In [78]:
logs = algobulls_connection.get_papertrading_logs(strategy_code2)
print(logs)

[2020-07-31 09:18:56] Performing sanity checks on cfg strategy_parameters, setting up broker connection and required data structures...
[2020-07-31 09:18:56] ABBroker connection has been setup successfully.
[2020-07-31 09:18:56] Sanity checks on cfg successful.
[2020-07-31 09:18:56] Setting up broker connection...
[2020-07-31 09:18:56] Broker connection has been setup successfully.
[2020-07-31 09:18:56] (NSE_EQ) Funds available in client's ABVirtualBroker account is : Rs. '1000000000.00'
[2020-07-31 09:18:56] 
########################################
 INITIALIZING ALGOBULLS CORE (v3.2.0 SECURE MODE)... 
########################################
[2020-07-31 09:18:56] Welcome ALGOBULLS VIRTUAL USER!
[2020-07-31 09:18:56] Reading strategy...
[2020-07-31 09:18:56] Entering Paper Trading mode. Henceforth, all timestamps will be Paper Trading timestamps...
[2020-07-31 09:18:56] STARTING ALGOBULLS CORE...
[2020-07-31 09:18:56] 
            
  #####  #######    #    ######  ####### ### #     # 

In [42]:
logs = algobulls_connection.get_papertrading_logs(strategy_code2)
print(logs)

[2020-07-31 14:48:51] Logs not available yet. Please retry in sometime.


### Recipe 10: MACD-Bracket-Order Strategy: Fetching Paper Trading Report - Profit-&-Loss Table

In [43]:
algobulls_connection.get_papertrading_report_pnl_table(strategy_code2)

Report not available yet. Please retry in sometime


### Recipe 11: MACD-Bracket-Order Strategy: Fetching Paper Trading Report - Statistics Table

In [44]:
algobulls_connection.get_papertrading_report_statistics(strategy_code2)

Report not available yet. Please retry in sometime


### Recipe 12: MACD-Bracket-Order Strategy: Fetching Paper Trading Report - Order History

In [45]:
order_history = algobulls_connection.get_papertrading_report_order_history(strategy_code2)
print(order_history)

[2020-07-31 14:48:54] Order History not available yet. Please retry in sometime.
